In [15]:

from datetime import datetime
import functools
import os

from IPython.display import HTML, clear_output

import gymnasium as gym
import jax
import jax.numpy as jnp
import matplotlib.pyplot as plt

try:
  import brax
except ImportError:
  !pip install git+https://github.com/google/brax.git@main
  clear_output()
  import brax

from brax import envs
from brax.envs import env as brax_env
from brax import jumpy as jp
from brax.io import html
from brax.io import model
from brax.training.agents.ppo import train as ppo
from brax.training.agents.sac import train as sac

if 'COLAB_TPU_ADDR' in os.environ:
  from jax.tools import colab_tpu
  colab_tpu.setup_tpu()

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
env

In [13]:
class FetchSlideEnv(env.Env):
    def __init__(self):
        self.gym_env = gym.make('FetchSlide-v2', max_episode_steps=10)

        # Define the observation space, action space, and reward function
        self.observation_space = self.gym_env.observation_space
        self.action_space = self.gym_env.action_space
        self.reward_function = self.gym_env.reward_function

    def step(self, action):
        # Use the step function from the gym environment
        observation, reward, done, info = self.gym_env.step(action)

        # Return the observation, reward, done, and info
        return observation, reward, done, info

    def reset(self):
        # Use the reset function from the gym environment
        observation = self.gym_env.reset()

        # Return the observation
        return observation

#### Enviornment

In [14]:
env = FetchSlideEnv()

AttributeError: 'MujocoFetchSlideEnv' object has no attribute 'reward_function'

In [10]:
env.reset(rng=jp.random_prngkey(seed=0))

TypeError: BaseRobotEnv.reset() got an unexpected keyword argument 'rng'

In [9]:
state = env.reset(rng=jp.random_prngkey(seed=0))

HTML(html.render(env.sys, [state.qp]))

TypeError: BaseRobotEnv.reset() got an unexpected keyword argument 'rng'